# Welcome to BASS!

One Click HRV

This notebok was custom designed by Abigail Dobyns for Dr. Curras-Collazo's lab.

    BASS: Biomedical Analysis Software Suite for event detection and signal processing.
    Copyright (C) 2015  Abigail Dobyns

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>

## Initalize

Run the following code block to intialize the program.

*run this block* **one** *time*

In [1]:
from bass import *

BASS ready!


# Begin User Input

For help, check out the wiki: [Protocol](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol)

Or the video tutorial: Coming Soon!

## Load Data File
Use the following block to change your settings. You **must** use this block.

Here are some helpful information about the loading settings:

**Settings['folder']= Full File Path for data input:**
Designate the path to your file to load. It can also be the relative path to the folder where this notebook is stored. This does not include the file itself.

    Mac OSX Example: '/Users/MYNAME/Documents/bass'
    Microsoft Example: r'C:\\Users\MYNAME\Documents\bass'
    

**Settings['Label']= File name:**
This is the name of your data file. It should include the file type. This file should NOT have a header and the first column must be time in seconds. Note: This file name will also appear as part of the output files names.

    'rat34_ECG.txt' 

**Settings['Output Folder'] = Full File Path for data output:** Designate the location of the folder where you would like the folder containing your results to go. If the folder does not exist, then it will be created. A plots folder, called 'plots' will be created inside this folder for you if it does not already exist. 

    Mac OSX Example: '/Users/MYNAME/Documents/output'
    Microsoft Example: r'C:\\Users\MYNAME\Documents\output' 

[Loading a file](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#load)

**WARNING** All text input should be raw, especially if in Windows.
    
    r'string!'
    r"string!"
    
## Other Settings

For more information about other settings, go to: 

[Transforming Data](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#enter-your-settings-for-data-transformation)

[Baseline Settings](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#set-baseline-for-thresholding)

[Peak Detection Settings](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#peak-detection-settings)

[Burst Detection Settings](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#enter-burst-settings)

In [94]:
#initialize new file
Data = {}
Settings = {}
Results ={}

############################################################################################
#manual Setting block
Settings['folder']= r"/Users/abigaildobyns/Desktop"
Settings['Label'] = r'pleth.txt'
Settings['Output Folder'] = r"/Users/abigaildobyns/Dropbox/Wilson_lab_papers/Abby/Pleth Analysis/Demo"

#transformation Settings
Settings['Absolute Value'] = True #Must be True if Savitzky-Golay is being used

Settings['Bandpass Highcut'] = False #in Hz
Settings['Bandpass Lowcut'] = False #in Hz
Settings['Bandpass Polynomial'] = False #integer

Settings['Linear Fit'] = False #between 0 and 1 on the whole time series
Settings['Linear Fit-Rolling R'] = 0.75 #between 0 and 1
Settings['Linear Fit-Rolling Window'] = 1000 #window for rolling mean for fit, unit is index not time
Settings['Relative Baseline'] = 0 #default 0, unless data is normalized, then 1.0. Can be any float

Settings['Savitzky-Golay Polynomial'] = 4 #integer
Settings['Savitzky-Golay Window Size'] = 25 #must be odd. units are index not time

#Baseline Settings
Settings['Baseline Type'] = r'rolling' #'linear', 'rolling', or 'static'
#For Linear
Settings['Baseline Start'] = 0.0 #start time in seconds
Settings['Baseline Stop'] = 1.0 #end time in seconds
#For Rolling
Settings['Rolling Baseline Window'] = 0.15 #leave as 'none' if linear or static

#Peaks
Settings['Delta'] = 2
Settings['Peak Minimum'] = 10 #amplitude value
Settings['Peak Maximum'] = 35  #amplitude value

#Bursts
Settings['Apnea Factor'] = 2 #factor to define apneas as a function of expiration
Settings['Burst Area'] = True #calculate burst area
Settings['Exclude Edges'] = True #False to keep edges, True to discard them
Settings['Inter-event interval minimum (seconds)'] = 0.0200 #only for bursts, not for peaks
Settings['Maximum Burst Duration (s)'] = 0.25 
Settings['Minimum Burst Duration (s)'] = 0.01
Settings['Minimum Peak Number'] = 1 #minimum number of peaks/burst, integer
Settings['Threshold']= 0.3 #linear: proportion of baseline. 
                           #static: literal value.
                           #rolling, linear ammount grater than rolling baseline at each time point.

#Outputs
Settings['Generate Graphs'] = False #create and save the fancy graph outputs


#Settings that you should not change unless you are a super advanced user:
#These are settings that are still in development
Settings['Graph LCpro events'] = False
Settings['File Type'] = r'Plain' #'LCPro', 'ImageJ', 'SIMA', 'Plain', 'Morgan'
Settings['Milliseconds'] = False
############################################################################################
##DO NOT TYPE BELOW THIS LINE
############################################################################################

def analyze(Data, Settings, Results):
    """
    The pipeline for event detection. Follows the strict '3 arguments in, 3 arguments out'
    rule. These dictionaries must be intalized before this, as well as all of the Settings values.
    Detects bursts and peaks for the data file that is uploaded.
    Settings used are saved out automatically with a time stamp as a receipt of each analysis performed.
    This version of analyze was made specifically for the UCSD HRV group.

    Parameters
    ----------
    Data: dictionary
        an empty dictionary named Data.
    Settings: dictionary
        dictionary that contains the user's settings.
    Results: dictionary
        an empty dictionary named Results.
    
    Returns
    -------
    Data: dictionary
        Contains the DataFrames with the time series data. Keys define which version of the data it is.
    Settings: dictionary
        dictionary that contains the user's settings.
    Results: dictionary
        Contains the following objects:
            Peaks: dictionary
                keys are the column names from the Data DataFrames. objects are DataFrames that contain information about each peak detected, indexed by peak time.
            Peaks-Master: DataFrame
                multi-indexed DataFrame, created by concatenating all Peaks DataFrames. Column names and peak time are the two indexes. Automatically saved in the Settings['output folder'] location.
            Bursts: dictionary
                keys are the column names from the Data DataFrames. objects are DataFrames that contain information about each burst detected. has an arbitrary index, which can be roughly thought of as burst number.
            Bursts-Master: DataFrame
                multi-indexed DataFrame, created by concatenating all Bursts DataFrames. Column names and burst number are the two indexes. Automatically saved in the Settings['output folder'] location.
            

    Notes
    -----
    This function is the top level function of the bass pipeline. 
    It has a few handy printed outputs, such as how long an analysis took, which step was just completed, lists of which objects contained no peaks or bursts. it also prints a list of key names and analysis measurements, which can be used in further analysis steps.
    
    """
    start = t.clock()
    #Load
    Data, Settings = load_wrapper(Data, Settings)
    
    Data['original'].columns = [Settings['Label']]
    
    #For this data set, we know the frame rate
    frames = np.round(1/Settings['Sample Rate (s/frame)'])
    Settings['Sample Rate (s/frame)'] = 1/float(frames)
    print "Rounded Sampling Rate (s/frame): %s" %Settings['Sample Rate (s/frame)']

    #transform data
    Data, Settings = transform_wrapper(Data, Settings)
    print 'Transformation completed'

    #set baseline
    Data, Settings, Results = baseline_wrapper(Data, Settings, Results)
    print 'Baseline set completed'

    #run peak detection
    Results = event_peakdet_wrapper(Data, Settings, Results)
    print 'Peak Detection completed'

    #run burst detection
    Results = event_burstdet_wrapper(Data, Settings, Results)
    print 'Burst Detection completed'

    #Save all the graphs
    if Settings['Generate Graphs'] == True:
        for label, col in Data['original'].iteritems():
            graph_detected_events_save(Data, Settings, Results, 
                                  roi = label, lcpro = Settings['Graph LCpro events'])
        print "Graphs Saved"

    #Save master files 
    Results['Peaks-Master'].to_csv(r'%s/%s_Peak_Results.csv'
                                   %(Settings['Output Folder'], Settings['Label']))
    Results['Bursts-Master'].to_csv(r'%s/%s_Bursts_Results.csv'
                                    %(Settings['Output Folder'], Settings['Label']))

    #Save Master Summary Files
    burst_grouped = Results['Bursts-Master'].groupby(level=0)
    burst_grouped = burst_grouped.describe()
    burst_grouped.to_csv(r'%s/%s_Bursts_Results_Summary.csv'
                                           %(Settings['Output Folder'], Settings['Label']))
    
    peak_grouped = Results['Peaks-Master'].groupby(level=0)
    peak_grouped= peak_grouped.describe()
    peak_grouped.to_csv(r'%s/%s_Peaks_Results_Summary.csv'
                                           %(Settings['Output Folder'], Settings['Label']))

    #save settings
    Settings_panda = DataFrame.from_dict(Settings, orient='index')
    now = datetime.datetime.now()
    colname = 'Settings: ' + str(now)
    Settings_panda.columns = [colname]
    Settings_panda = Settings_panda.sort()
    Settings_panda.to_csv(r"%s/%s_Settings_%s.csv"%(Settings['Output Folder'], 
                                                 Settings['Label'], 
                                                 now.strftime('%Y_%m_%d__%H_%M_%S')))

    end = t.clock()
    run_time = end-start
    print "Analysis Complete: ", np.round(run_time,4), " Seconds"
    
    print "\n--------------------------------------------"

    print "Data Column Names/Keys"
    print "-----"
    for name in Data['original']:
        print name
    print "\n--------------------------------------------"
    print "Available Measurements from Peaks for further analysis:"
    print "-----"
    for label, col in Results['Peaks-Master'].iteritems():
        print label
    print "\n--------------------------------------------"
    print "Available Measurements from Bursts for further analysis:"
    print "-----"
    for label, col in Results['Bursts-Master'].iteritems():
        print label
    
    print "\n---------------------------"
    print '|Event Detection Complete!|'
    print "---------------------------\n"
    return Data, Settings, Results

pd.options.display.max_columns = 25

#Run detection
Data, Settings, Results = analyze(Data, Settings, Results)

#New pleth stuff

key = Settings['Label']
start_time = t.clock()
#create results table
pleth = pd.DataFrame(columns=['Breaths', 'Recording Length (s)','Mean Breath Rate', 
                              'AUC', 'AUC STD', 'Insp Time mean', 'Insp Time std',
                              'Exp Time mean', 'Exp Time mean', 'TTotal mean', 'TTotal std',
                              'Apnea Count', 'TI Samp Ent', 'TE Samp Ent', 'TTot Samp Ent',
                             'TI Hist Ent', 'TE Hist Ent', 'TTot Hist Ent'],
                  index = [key])
key = Settings['Label']
event_type = 'Bursts'
#total number of breaths
try:
    pleth.ix[key]['Breaths'] = Results['Bursts'][key]['Burst Duration'].count()
    
except:
    pleth.ix[key]['Breaths'] = NaN

#length    
try:
    t_sec = Data['trans'].index[-1]-Data['trans'].index[0]
    pleth.ix[key]['Recording Length (s)'] = t_sec
except:
    pleth.ix[key]['Recording Length (s)'] = NaN
#breath rate   
try:
    t_min = t_sec/60
    pleth.ix[key]['Mean Breath Rate'] = pleth.ix[key]['Breaths']/t_min
except:
    pleth.ix[key]['Mean Breath Rate'] = NaN
#Area under the curve
try:
    pleth.ix[key]['AUC'] = Results['Bursts'][key]['Burst Area'].mean()
    pleth.ix[key]['AUC STD'] = Results['Bursts'][key]['Burst Area'].std()
except:
    pleth.ix[key]['AUC'] = NaN
    pleth.ix[key]['AUC'] = NaN
#inspiration    
try:
    pleth.ix[key]['Insp Time mean'] = Results['Bursts'][key]['Attack'].mean()
    pleth.ix[key]['Insp Time std'] = Results['Bursts'][key]['Attack'].std()
except:
    pleth.ix[key]['Insp Time mean'] = NaN
    pleth.ix[key]['Insp Time std'] = NaN
#Expiration
try:
    pleth.ix[key]['Exp Time mean'] = Results['Bursts'][key]['Decay'].mean()
    pleth.ix[key]['Exp Time std'] = Results['Bursts'][key]['Decay'].std()
except:
    pleth.ix[key]['Exp Time mean'] = NaN
    pleth.ix[key]['Exp Time std'] = NaN
#TTOTAL
try:
    pleth.ix[key]['TTotal mean'] = Results['Bursts'][key]['Total Cycle Time'].mean()
    pleth.ix[key]['TTotal std'] = Results['Bursts'][key]['Total Cycle Time'].std()
except:
    pleth.ix[key]['TTotal mean'] = NaN
    pleth.ix[key]['TTotal std'] = NaN

#apnea
try:
    apnea = Settings['Apnea Factor']*(Results['Bursts'][key]['Decay'].mean())
    temp = Results['Bursts'][key]['Interburst Interval']
    apnea_count = float(temp[temp>apnea].count())
    pleth.ix[key]['Apnea Count'] = apnea_count
except:
    pleth.ix[key]['Apnea Count'] = NaN

#Shannon Entropy
try:
    meas = 'Attack'
    Results = samp_entropy_wrapper(event_type, meas, Data, Settings, Results)
    pleth.ix[key]['TI Samp Ent'] = float(Results['Sample Entropy'][meas])
except:
    pleth.ix[key]['TI Samp Ent'] = NaN

try:
    meas = 'Decay'
    Results = samp_entropy_wrapper(event_type, meas, Data, Settings, Results)
    pleth.ix[key]['TE Samp Ent'] = float(Results['Sample Entropy'][meas])
except:
    pleth.ix[key]['TE Samp Ent'] = NaN
    
try:
    meas = 'Total Cycle Time'
    Results = samp_entropy_wrapper(event_type, meas, Data, Settings, Results)
    pleth.ix[key]['TTot Samp Ent'] = float(Results['Sample Entropy'][meas])
except:
    pleth.ix[key]['TTot Samp Ent'] = NaN

#poincare
try:
    meas = 'Total Cycle Time'
    Results = poincare_batch(event_type, meas, Data, Settings, Results)
    meas = 'Attack'
    Results = poincare_batch(event_type, meas, Data, Settings, Results)
    meas = 'Decay'
    Results = poincare_batch(event_type, meas, Data, Settings, Results)
except:
    print "Poincare Failed"

#Hist Ent
try:
    meas = 'all'
    Results = histent_wrapper(event_type, meas, Data, Settings, Results)
    pleth.ix[key]['TI Hist Ent'] = float(Results['Histogram Entropy']['Attack'])
    pleth.ix[key]['TE Hist Ent'] = float(Results['Histogram Entropy']['Decay'])
    pleth.ix[key]['TTot Hist Ent'] = float(Results['Histogram Entropy']['Total Cycle Time'])
    
except:
    print "Histogram Entropy Failed"
    pleth.ix[key]['TI Hist Ent'] = NaN
    pleth.ix[key]['TE Hist Ent'] = NaN
    pleth.ix[key]['TTot Hist Ent'] = NaN

try:
    #Moving Stats
    event_type = 'Bursts'
    meas = 'Total Cycle Time'
    window = 30 #seconds
    Results = moving_statistics(event_type, meas, window, Data, Settings, Results)
except:
    pass

pleth.to_csv(r"%s/%s_Pleth.csv"%(Settings['Output Folder'],Settings['Label']))
end_time = t.clock()

print 'Heart Rate Varibility Analysis Complete: %s sec' %np.round((end_time- start_time), 4)
pleth

Made plots folder
Data Loaded
Rounded Sampling Rate (s/frame): 0.00025
pleth.txt is 330.0 seconds long.
Rounded Sampling Rate (s/frame): 0.00025
Transformation completed
Baseline set completed
Peak Detection completed
Burst Detection completed
Analysis Complete:  56.3853  Seconds

--------------------------------------------
Data Column Names/Keys
-----
pleth.txt

--------------------------------------------
Available Measurements from Peaks for further analysis:
-----
Peaks Amplitude
Intervals

--------------------------------------------
Available Measurements from Bursts for further analysis:
-----
Burst Start
Burst End
Burst Duration
Burst Start Amplitude
Burst End Amplitude
Edge Event
Interburst Interval
Total Cycle Time
Peaks per Burst
Peak Amp
Peak Time
Attack
Decay
Intraburst Frequency
Burst Area

---------------------------
|Event Detection Complete!|
---------------------------

All Bursts measurements analyzed.
Total Cycle Time Count
     pleth.txt
0            0
30         

,Breaths,Recording Length (s),Mean Breath Rate,AUC,AUC STD,Insp Time mean,Insp Time std,Exp Time mean,Exp Time mean,TTotal mean,TTotal std,Apnea Count,TI Samp Ent,TE Samp Ent,TTot Samp Ent,TI Hist Ent,TE Hist Ent,TTot Hist Ent
pleth.txt,1546,330,281.0909,1.003672,0.2587953,0.02200194,0.007623179,0.02095893,0.02095893,0.1699049,0.1300218,1527,1.694886,1.770121,0.7968959,0.6714096,0.5358827,0.5206133


#OPTIONAL GRAPHS AND ANALYSIS

The following blocks are optional calls to other figures and analysis

## Display Event Detection Tables
#### Display Settings used for analysis

In [ ]:
display_settings(Settings)

#### Display Summary Results for Peaks

In [3]:
#grouped summary for peaks
Results['Peaks-Master'].groupby(level=0).describe()

Peaks Amplitude    Intervals
pleth.txt count      1547.000000  1546.000000
          mean         24.302622     0.212610
          std           2.397677     0.184269
          min          17.652616     0.067500
          25%          22.553240     0.105000
          50%          24.256364     0.145000
          75%          25.861566     0.247687
          max          33.351401     1.433500

#### Display Summary Results for Bursts

In [4]:
#grouped summary for bursts
Results['Bursts-Master'].groupby(level=0).describe()

Burst Start    Burst End  Burst Duration  \
pleth.txt count  1546.000000  1546.000000     1546.000000   
          mean    712.718100   712.761061        0.042961   
          std      96.529026    96.529403        0.009837   
          min     550.158500   550.189500        0.024250   
          25%     625.359438   625.402875        0.036250   
          50%     718.922625   718.965250        0.041750   
          75%     793.834625   793.892063        0.047937   
          max     878.754750   878.801750        0.142000   

                 Burst Start Amplitude  Burst End Amplitude Edge Event  \
pleth.txt count            1546.000000          1546.000000       1546   
          mean               22.206638            22.265840          0   
          std                 2.261431             2.263390          0   
          min                16.187657            16.193442      False   
          25%                20.480524            20.513925          0   
          50%                22.152867            22.191023          0   
          75%                23.923696            23.988946          0   
          max                29.559500            29.884656      False   

                 Interburst Interval  Total Cycle Time  Peaks per Burst  \
pleth.txt count          1546.000000       1546.000000             1546   
          mean              0.126944          0.169905                1   
          std               0.127611          0.130022                0   
          min               0.026500          0.065750                1   
          25%               0.062500          0.102500                1   
          50%               0.082625          0.128000                1   
          75%               0.135750          0.181000                1   
          max               1.097500          1.140750                1   

                    Peak Amp    Peak Time       Attack        Decay  \
pleth.txt count  1546.000000  1546.000000  1546.000000  1546.000000   
          mean     24.300733   712.740102     0.022002     0.020959   
          std       2.397302    96.529548     0.007623     0.006380   
          min      17.652616   550.172750     0.004250     0.007750   
          25%      22.551873   625.378375     0.016750     0.017250   
          50%      24.255340   718.945875     0.020750     0.020250   
          75%      25.861106   793.866688     0.025500     0.023750   
          max      33.351401   878.784000     0.077750     0.091250   

                 Intraburst Frequency   Burst Area  
pleth.txt count           1546.000000  1546.000000  
          mean              24.318109     1.003672  
          std                4.887472     0.258795  
          min                7.042254     0.468373  
          25%               20.860602     0.834109  
          50%               23.952096     0.977661  
          75%               27.586207     1.123621  
          max               41.237113     3.236941

# Interactive Graphs
## Line Graphs
####One pannel, detected events
Plot one time series by calling it's name

In [5]:
#Interactive, single time series by Key
key = Settings['Label']
graph_ts(Data, Settings, Results, key)

####Two pannel
Create line plots of the raw data as well as the data analysis. 

Plots are saved by clicking the save button in the pop-up window with your graph.

    key = 'Mean1'
    start =100 
    end= 101

[Results Line Plot](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#line-plots)

In [5]:
key = Settings['Label']
start =100 #start time in seconds
end= 101#end time in seconds
results_timeseries_plot(key, start, end, Data, Settings, Results)

## Autocorrelation

Display the Autocorrelation plot of your transformed data.

Choose the start and end time in seconds. to capture whole time series, use end = -1. May be slow

    key = 'Mean1'
    start = 0 
    end = 10
    
[Autocorrelation Plot](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#autocorrelation)

In [ ]:
#autocorrelation
key = Settings['Label']
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end.
autocorrelation_plot(Data['trans'][key][start:end])
plt.show()

## Raster Plot

Shows the temporal relationship of peaks in each column. Auto scales. Display only. Intended for more than one column of data

In [ ]:
#raster
raster(Data, Results)

## Frequency Plot
Use this block to plot changes of any measurement over time. Does not support 'all'. Example:

    event_type = 'Peaks'
    meas = 'Intervals'
    key = 'Mean1'

[Frequency Plot](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#frequency-plot)

In [5]:
event_type = 'Peaks'
meas = 'Intervals'
key = Settings['Label']
frequency_plot(event_type, meas, key, Data, Settings, Results)

## Analyze Events by Measurement
Generates a line plot with error bars for a given event measurement. X axis is the names of each time series. Display Only. Intended for more than one column of data. This is not a box and whiskers plot.

    event_type = 'peaks'
    meas = 'Peaks Amplitude'
    
[Analyze Events by Measurement](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#analyze-events-by-measurement)

In [ ]:
#Get average plots, display only
event_type = 'Peaks'
meas = 'Intervals'
average_measurement_plot(event_type, meas,Results)

## Poincare Plots

Create a Poincare Plot of your favorite varible. Choose an event type (Peaks or Bursts), measurement type. Calling meas = 'All' is supported.

Plots and tables are saved automatically

Example:

    event_type = 'Bursts'
    meas = 'Burst Duration'

[More on Poincare Plots](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#poincare-plots)

####Batch Poincare

[Batch Poincare](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#batch-poincare)

In [81]:
#Batch
event_type = 'Bursts'
meas = 'Total Cycle Time'
Results = poincare_batch(event_type, meas, Data, Settings, Results)
pd.concat({'SD1':Results['Poincare SD1'],'SD2':Results['Poincare SD2']})

,,Attack,Decay,Total Cycle Time
SD1,pleth.txt,0.005717,0.005968,0.117689
SD2,pleth.txt,0.009136,0.006769,0.140370


####Quick Poincare Plot

Quickly call one poincare plot for display. Plot and Table are not saved automatically. Choose an event type (Peaks or Bursts), measurement type, and key. Calling meas = 'All' is not supported.

[Quick Poincare](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#quick-poincare-plots)

In [77]:
#quick
event_type = 'Bursts'
meas = 'Attack'
key = Settings['Label']
poincare_plot(Results[event_type][key][meas])

Attack results:
SD1 = 0.0057 s
SD2 = 0.0091 s


## Power Spectral Density
The following blocks allows you to asses the power of event measuments in the frequency domain. While you can call this block on any event measurement, it is intended to be used on interval data (or at least data with units in seconds). Reccomended:

    event_type = 'Bursts'
    meas = 'Total Cycle Time'
    key = 'Mean1'
    scale = 'raw'
    
    event_type = 'Peaks'
    meas = 'Intervals'
    key = 'Mean1'
    scale = 'raw'

Because this data is in the frequency domain, we must interpolate it in order to perform a FFT on it. Does not support 'all'.

[Power Spectral Density: Events](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#power-spectral-density)

### Events

Use the code block below to specify your settings for event measurment PSD.

In [ ]:
Settings['PSD-Event'] = Series(index = ['Hz','ULF', 'VLF', 'LF','HF','dx'])
#Set PSD ranges for power in band

Settings['PSD-Event']['hz'] = 4.0 #freqency that the interpolation and PSD are performed with.
Settings['PSD-Event']['ULF'] = 0.03 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Event']['VLF'] = 0.05 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Event']['LF'] = 0.15 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Event']['HF'] = 0.4 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2)
Settings['PSD-Event']['dx'] = 10 #segmentation for the area under the curve. 

In [ ]:
event_type = 'Peaks'
meas = 'Intervals'
key = Settings['Label']
scale = 'raw'
Results = psd_event(event_type, meas, key, scale, Data, Settings, Results)
Results['PSD-Event'][key]

### Time Series

Use the settings code block to set your frequency bands to calculate area under the curve. This block is not required. band output is always in raw power, even if the graph scale is dB/Hz.

[Power Spectral Density: Signal](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#power-spectral-density-optional)

In [ ]:
#optional
Settings['PSD-Signal'] = Series(index = ['ULF', 'VLF', 'LF','HF','dx'])

#Set PSD ranges for power in band
Settings['PSD-Signal']['ULF'] = 25 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Signal']['VLF'] = 75 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Signal']['LF'] = 150 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Signal']['HF'] = 300 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2) where hz is the sampling frequency
Settings['PSD-Signal']['dx'] = 2 #segmentation for integration of the area under the curve. 

Use the block below to generate the PSD graph and power in bands results (if selected). scale toggles which units to use for the graph:
    
    raw = s^2/Hz
    db = dB/Hz = 10*log10(s^2/Hz)
    
Graph and table are automatically saved in the `PSD-Signal` subfolder.

In [ ]:
scale = 'raw' #raw or db
Results = psd_signal(version = 'original', key = 'Mean1', scale = scale, 
                     Data = Data, Settings = Settings, Results = Results)
Results['PSD-Signal']

### Spectrogram

Use the block below to get the spectrogram of the signal. The frequency (y-axis) scales automatically to only show 'active' frequencies. This can take some time to run. 

    version = 'original'
    key = 'Mean1'

After transformation is run, you can call version = 'trans'. This graph is not automatically saved.

[Spectrogram](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#spectrogram)

In [ ]:
version = 'original'
key = Settings['Label']
spectogram(version, key, Data, Settings, Results)

## Descriptive Statistics

#### Moving/Sliding Averages, Standard Deviation, and Count
Generates the moving **mean**, **standard deviation**, and **count** for a given measurement across all columns of the Data in the form of a DataFrame (displayed as a table).
Saves out the dataframes of these three results automatically with the window size in the name as a .csv.
If **meas == 'All'**, then the function will loop and produce these tables for all measurements.

    event_type = 'Peaks'
    meas = 'all'
    window = 30
    
[Moving Stats](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#analyze-events-by-measurement)

In [93]:
#Moving Stats
event_type = 'Bursts'
meas = 'Total Cycle Time'
window = 30 #seconds
Results = moving_statistics(event_type, meas, window, Data, Settings, Results)

Total Cycle Time Count
     pleth.txt
0            0
30           0
60           0
90           0
120          0
150          0
180          0
210          0
240          0
270          0
300          0
330          0
360          0
390          0
420          0
450          0
480          0
510          0
540        119
570        166
600        119
630         99
660        128
690        146
720        208
750        122
780        116
810        114
840        170
Total Cycle Time Mean
     pleth.txt
0     0.000000
30    0.000000
60    0.000000
90    0.000000
120   0.000000
150   0.000000
180   0.000000
210   0.000000
240   0.000000
270   0.000000
300   0.000000
330   0.000000
360   0.000000
390   0.000000
420   0.000000
450   0.000000
480   0.000000
510   0.000000
540   0.140704
570   0.149724
600   0.217416
630   0.229639
660   0.201664
690   0.163940
720   0.133573
750   0.184018
780   0.170933
810   0.159822
840   0.155459
Total Cycle Time Std
     pleth.txt
0     0.000000
30  

## Entropy

### Histogram Entropy
Calculates the histogram entropy of a measurement for each column of data. Also saves the histogram of each. If meas is set to 'all', then all available measurements from the event_type chosen will be calculated iteratevely. 

If all of the samples fall in one bin regardless of the bin size, it means we have the most predictable sitution and the entropy is 0. If we have uniformly dist function, the max entropy will be 1

    event_type = 'Bursts'
    meas = 'all'

[Histogram Entropy](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#histentropy)

In [82]:
#Histogram Entropy
event_type = 'Bursts'
meas = 'all'
Results = histent_wrapper(event_type, meas, Data, Settings, Results)
Results['Histogram Entropy']

All Bursts measurements analyzed.


,Burst Start,Burst End,Burst Duration,Burst Start Amplitude,Burst End Amplitude,Edge Event,Interburst Interval,Total Cycle Time,Peaks per Burst,Peak Amp,Peak Time,Attack,Decay,Intraburst Frequency,Burst Area
pleth.txt,0.973718,0.973718,0.590168,0.858843,0.850922,0,0.505385,0.520613,0,0.827021,0.973718,0.67141,0.535883,0.805133,0.630351


## Approximate entropy
this only runs if you have pyeeg.py in the same folder as this notebook and bass.py. **WARNING: THIS FUNCTION RUNS SLOWLY**

run the below code to get the approximate entropy of any measurement or raw signal. Returns the entropy of the entire results array (no windowing). I am using the following M and R values:

    M = 2  
    R = 0.2*std(measurement)
    
these values can be modified in the source code. alternatively, you can call ap_entropy directly. supports 'all'

**Interpretation**: A time series containing many repetitive patterns has a relatively small ApEn; a less predictable process has a higher ApEn.

[Approximate Entropy in BASS](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#approximate-entropy)

[Aproximate Entropy Source](http://pyeeg.sourceforge.net/)

###Events

In [ ]:
#Approximate Entropy
event_type = 'Peaks'
meas = 'Intervals'
Results = ap_entropy_wrapper(event_type, meas, Data, Settings, Results)
Results['Approximate Entropy']

###Time Series

In [ ]:
#Approximate Entropy on raw signal
#takes a VERY long time
from pyeeg import ap_entropy

version = 'original' #original, trans, shift, or rolling
key = Settings['Label'] #Mean1 default key for one time series
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

ap_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

### Sample Entropy
this only runs if you have pyeeg.py in the same folder as this notebook and bass.py. **WARNING: THIS FUNCTION RUNS SLOWLY**

run the below code to get the sample entropy of any measurement. Returns the entropy of the entire results array (no windowing). I am using the following M and R values:

    M = 2  
    R = 0.2*std(measurement)
    
these values can be modified in the source code. alternatively, you can call samp_entropy directly. 
Supports 'all'

[Sample Entropy in BASS](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#sample-entropy)

[Sample Entropy Source](http://pyeeg.sourceforge.net/)

###Events

In [73]:
#Sample Entropy
event_type = 'Bursts'
meas = 'Total Cycle Time'
Results = samp_entropy_wrapper(event_type, meas, Data, Settings, Results)
Results['Sample Entropy']

,Attack,Decay,Total Cycle Time
pleth.txt,1.694886,1.770121,0.796896


In [74]:
Results['Sample Entropy']['Attack']

pleth.txt    1.694886
Name: Attack, dtype: float64

###Time Series

In [ ]:
#on raw signal
#takes a VERY long time
version = 'original' #original, trans, shift, or rolling
key = Settings['Label']
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

samp_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

# Helpful Stuff

While not completely up to date with some of the new changes, the Wiki can be useful if you have questions about some of the settings: https://github.com/drcgw/SWAN/wiki/Tutorial

# More Help?

Stuck on a particular step or function?
Try typing the function name followed by two ??. This will pop up the docstring and source code.
You can also call help() to have the notebook print the doc string.

    Example:
    analyze??
    help(analyze)

In [ ]:
help(moving_statistics)

In [ ]:
moving_statistics??

##Blank Code Block
you're still here, reading? you must be a dedicated super user!

If that is the case, then you must know how to code in Python. Use this space to get crazy with your own advanced analysis and stuff.

[Blank Code Block](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#blank-code-block)